In [ ]:
# --- Импорт необходимых библиотек ---
import pandas as pd  # Основная библиотека для работы с данными в формате таблиц (DataFrame)
import json          # Библиотека для работы с форматом JSON, нужна для сохранения данных
from pathlib import Path  # Удобный инструмент для работы с путями к файлам и папкам
from tqdm.auto import tqdm  # Библиотека для создания красивых и информативных прогресс-баров
import warnings      # Инструмент для управления предупреждениями Python

# Отключаем предупреждения типа FutureWarning. Это делается для чистоты вывода,
# так как некоторые библиотеки могут использовать устаревшие, но все еще работающие функции.
warnings.filterwarnings("ignore", category=FutureWarning)


# --- 1. КОНФИГУРАЦИЯ ---
# В этом блоке мы определяем все ключевые параметры, чтобы легко менять их в одном месте.

# Путь к исходному файлу с данными.
INPUT_FILE = "20250628_dm_by_clusters_train_preprocessed.parquet"

# Имя папки, куда будут сохранены подготовленные для обучения данные.
OUTPUT_DIR = Path("chronos_univariate_dataset")

# Имя столбца, который является нашей целевой переменной (то, что мы хотим прогнозировать).
TARGET = "sales_qty"

# ВАЖНО: Минимальная длина временного ряда, который мы будем использовать для обучения.
# ЗАЧЕМ ЭТО НУЖНО: Модель Chronos для обучения требует, чтобы у временного ряда была
# достаточная длина для "вырезания" как минимум одного обучающего примера.
# Обучающий пример состоит из `context` (история) и `prediction` (будущее).
# Поэтому минимальная длина должна быть как минимум `context_length + prediction_length`.
# Устанавливая здесь порог, мы заранее отфильтровываем слишком короткие ряды,
# экономя время на их обработке и избегая ошибок на этапе обучения.
MIN_SERIES_LENGTH = 102  # Например, для context=90, prediction=12 (90+12=102)


# --- ОСНОВНОЙ КОД ---
print("--- Шаг 1: Загрузка данных ---")
# Используем блок try-except для "безопасной" загрузки файла.
# Если файл не будет найден, программа выведет понятное сообщение об ошибке и завершится.
try:
    # pd.read_parquet - это высокопроизводительный способ чтения данных из формата Parquet.
    df = pd.read_parquet(INPUT_FILE)
    print(f"DataFrame успешно загружен. Форма: {df.shape}")
except FileNotFoundError:
    print(f"Ошибка: Файл '{INPUT_FILE}' не найден. Проверьте путь и имя файла.")
    exit() # Завершаем выполнение скрипта, так как без данных он бесполезен.

print("\n--- Шаг 2: Сохранение непрерывных временных рядов (на основе ClipID) в формате JSON Lines ---")
# Создаем папку для выходных данных, если она еще не существует.
# exist_ok=True предотвращает ошибку, если папка уже есть.
OUTPUT_DIR.mkdir(exist_ok=True)
# Собираем полный путь к файлу, в который будем записывать данные.
train_file = OUTPUT_DIR / "univariate_train.jsonl"

# Мы определили, что в ваших данных каждый уникальный непрерывный
# временной ряд идентифицируется тройкой: (product_id, cluster_id, clip_id).
# Поэтому мы группируем DataFrame именно по этим трем столбцам.
# Каждая 'group' в цикле ниже будет представлять собой один полный, непрерывный временной ряд.
grouped = df.groupby(['product_id', 'cluster_id', 'clip_id'])

print(f"Найдено {len(grouped)} уникальных непрерывных рядов (ClipID).")

# Открываем файл для записи. `with open(...)` гарантирует, что файл будет корректно закрыт
# даже в случае ошибки.
with open(train_file, 'w') as fp:
    # Итерируемся по каждой группе. `tqdm` оборачивает итератор, чтобы показать прогресс-бар.
    # `name` будет кортежем (product_id, cluster_id, clip_id), `group` - DataFrame с данными для этого ряда.
    for name, group in tqdm(grouped, desc="Формирование и сохранение временных рядов"):
        
        # На всякий случай сортируем данные внутри группы по временному индексу.
        # Это гарантирует, что последовательность данных корректна.
        group = group.sort_values('time_idx')
        
        # Извлекаем значения целевой переменной и преобразуем их в обычный список Python.
        target = group[TARGET].tolist()
        
        # --- ФИЛЬТРАЦИЯ ---
        # Проверяем, соответствует ли длина текущего непрерывного ряда нашему порогу.
        if len(target) < MIN_SERIES_LENGTH:
            continue # `continue` немедленно переходит к следующей итерации цикла, пропуская этот ряд.

        # --- ФОРМИРОВАНИЕ УНИКАЛЬНОГО ID И ЗАПИСИ ---
        # Распаковываем кортеж `name`, чтобы получить отдельные ID.
        product_id, cluster_id, clip_id = name
        # Создаем уникальный идентификатор для этого ряда, объединяя все три ID.
        # Это полезно для последующей отладки и анализа результатов.
        item_id = f"{product_id}_{cluster_id}_{clip_id}"

        # Создаем словарь - одну запись для нашего JSON Lines файла.
        # Формат ({'start': ..., 'target': ..., 'item_id': ...}) стандартен для библиотеки GluonTS,
        # на которой основан наш скрипт обучения.
        entry = {
            "start": 0,          # ПОЧЕМУ 0: Мы используем фиктивную числовую дату начала.
                                 # Это решает проблему совместимости с GluonTS, который
                                 # пытается парсить строки как даты, но для математических
                                 # операций (вычисления смещений) ему нужно число.
            "target": target,    # Список с данными о продажах.
            "item_id": item_id   # Уникальный идентификатор ряда.
        }
        
        # Преобразуем словарь в строку JSON и записываем ее в файл, добавляя символ переноса строки.
        fp.write(json.dumps(entry) + '\n')

print(f"\nГотово! Данные для одномерного fine-tuning'а сохранены в: {train_file}")
print("Каждый ClipID был обработан как отдельный временной ряд.")

--- Шаг 1: Загрузка данных ---
DataFrame успешно загружен. Форма: (14939782, 60)

--- Шаг 2: Сохранение непрерывных временных рядов (на основе ClipID) в формате JSON Lines ---
Найдено 182447 уникальных непрерывных рядов (ClipID).


Формирование и сохранение временных рядов:   0%|          | 0/182447 [00:00<?, ?it/s]


Готово! Данные для одномерного fine-tuning'а сохранены в: chronos_univariate_dataset\univariate_train.jsonl
Каждый ClipID был обработан как отдельный временной ряд.
